In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
cookie = {
  'laravel_session': '',
  'XSRF-TOKEN':  ''
}
class Scraper:
  def __init__(self, url, cookie):
    self.url = url
    self.cookie = cookie
  def get_page(self):
    page = requests.get(self.url, cookies=self.cookie)
    return page.text
  def get_soup(self):
    soup = BeautifulSoup(self.get_page(), 'html.parser')
    return soup
  def get_title(self):
    soup = self.get_soup()
    title = soup.title.string
    return title
  def get_all_links(self):
    soup = self.get_soup()
    links = soup.find_all('a')
    return links
  def get_table(self):
    soup = self.get_soup()
    table = soup.find_all('table')
    return table

scraper = Scraper('https://daotao.vku.udn.vn/sv/hoc-phan-con-lai', cookie)
table = scraper.get_table()

In [ ]:
print(table)

In [ ]:
class HocPhan:
  def __init__(self, ten, so_tin_chi):
    self.ten = ten
    self.so_tin_chi = so_tin_chi
  def __str__(self):
    return f'{self.ten} - {self.so_tin_chi}'

In [ ]:
arrHocPhan: list[HocPhan] = []
body = table[0].find('tbody')
for tr in body.find_all('tr'):
  tds = tr.find_all('td')
  ten = tds[1].text
  so_tin_chi = tds[4].find('code').text
  arrHocPhan.append(HocPhan(ten, so_tin_chi))

In [ ]:
import json
serialized_data = []
for hocphan in arrHocPhan:
    serialized_data.append({
        'ten': hocphan.ten,
        'so_tin_chi': hocphan.so_tin_chi
    })

filename = 'hocphan_data.json'
with open(filename, 'w', encoding='utf-8') as file:
    json.dump(serialized_data, file, ensure_ascii=False, indent=4)


In [ ]:
import csv

with open('daotao.csv', 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.writer(f)
    writer.writerow( ['Tên học phần', 'Số tín chỉ'])
    for hocPhan in arrHocPhan:
        writer.writerow([hocPhan.ten, hocPhan.so_tin_chi])
